In [1]:
using Revise
using Pkg; Pkg.activate(".")

using Dates
using Statistics # limited set of methods included in Julia Base
using StatsBase  # extends Statistics with more functions
using Rotations
# using SignalAlignment
using Interpolations
using PyPlot
using JLD2
# using MAT

  Activating project at `~/Projects/ASTRAL/lidar`


In [2]:
# rotations example

# RotXY(roll, pitch) # radians
# matrix that rotates around y (pitch), then by x (roll) in radians
RotXY(10/180*pi, 5/180*pi)

3×3 RotXY{Float64} with indices SOneTo(3)×SOneTo(3)(0.0872665, 0.174533):
  0.984808   0.0         0.173648
  0.0151344  0.996195   -0.0858317
 -0.172987   0.0871557   0.98106

For leg 1 we assume it's fully stabilized, so we can measure the difference between the heave at the lidar and at the POSMV.
The heave measured by the POSMV is assumed to be vertical, and the heave at the VectorNav on the lidar, is also vertical when stabilized.
In this case, the displacement moments $(L_x, L_y, L_z)$ between the lidar and the POSMV are responsible for the difference in heave vertical velocity at the lidar $w$ compared to the POSMV $w_0$,
$$
w' = w - w_0 = A\theta + B\phi,\\
A = (L_x^2 + L_z^2)^{1/2},\\
B = (L_y^2 + L_z^2)^{1/2}
$$
where $\theta$ is the pitch and $\phi$ is the roll. This is a regression problem for ${\bf a} = [A; B]$, given $w'$ and $v=[\theta; \phi]$.

$$
{\bf w'} = {\bf av}\\
{\bf a} = {\bf w'}{\bf v}^{-1} = {\bf v}\backslash {\bf w'}
$$

### Procedure
1. Synchronize POSMV pitch and VectorNav roll 